## HotChili Analytics trading notebook template
#### Configure by setting ALGO_NAME in cell below.
#### Run various options (backtest, ingest, live) by uncommenting one cell 

In [1]:
%matplotlib inline
%load_ext zipline

# %reload_ext zipline # Uncomment and use this when already loaded zipline extension for magic cell usage.

/home/hca-dev/miniconda3/envs/hca/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/home/hca-dev/miniconda3/envs/hca/lib/python3.6/site-packages/odo/backends/pandas.py:94: FutureWarning: pandas.tslib is deprecated and will be removed in a future version.
You can access NaTType as type(pandas.NaT)
  @convert.register((pd.Timestamp, pd.Timedelta), (pd.tslib.NaTType, type(None)))


In [2]:
import pandas as pd

# Options you can uncomment and set:

# pd.set_option("max_colwidth", 300)
# pd.set_option("display.max_rows", 300)
# pd.set_option("display.max_columns", 50)
# pd.set_option('precision', 2)
# pd.options.display.float_format = '{:20,.2f}'.format

In [3]:
import os

hca_root_path = os.environ['HCA_ROOT']
print(f"hca_root_path = {hca_root_path}")

hca_root_path = /home/hca-ws002/hca


# Construct algorithm strategy path names

Assumptions:

- the strategy is in a directory with the same name as the strategy in `ALGO_NAME` below
- the strategy is located in the hca-resources directory, which is located relative to `hca_root_path`, found above

In [4]:
ALGO_NAME = "HCA_Fixed_Ratio_Allocations" # <--- Supply name here

Other variables are derived from `ALGO_NAME`:

In [5]:
HCA_RESOURCES_PATH = hca_root_path + "/hca-resources/" 
ALGO_PATH          = HCA_RESOURCES_PATH + ALGO_NAME + "/" 
ALGO_BT            = ALGO_PATH + ALGO_NAME + ".py"
ALGO_BT_OUT        = ALGO_PATH + ALGO_NAME + ".pkl"
ALGO_LIVE          = ALGO_PATH + ALGO_NAME + "_Live" + ".py"

print(f"""
ALGO_NAME          = {ALGO_NAME}
HCA_RESOURCES_PATH = {HCA_RESOURCES_PATH}
ALGO_PATH          = {ALGO_PATH}
ALGO_BT            = {ALGO_BT}
ALGO_BT_OUT        = {ALGO_BT_OUT}
ALGO_LIVE          = {ALGO_LIVE}

Contents of algo directory:
""")

!ls $ALGO_PATH


ALGO_NAME          = HCA_Fixed_Ratio_Allocations
HCA_RESOURCES_PATH = /home/hca-ws002/hca/hca-resources/
ALGO_PATH          = /home/hca-ws002/hca/hca-resources/HCA_Fixed_Ratio_Allocations/
ALGO_BT            = /home/hca-ws002/hca/hca-resources/HCA_Fixed_Ratio_Allocations/HCA_Fixed_Ratio_Allocations.py
ALGO_BT_OUT        = /home/hca-ws002/hca/hca-resources/HCA_Fixed_Ratio_Allocations/HCA_Fixed_Ratio_Allocations.pkl
ALGO_LIVE          = /home/hca-ws002/hca/hca-resources/HCA_Fixed_Ratio_Allocations/HCA_Fixed_Ratio_Allocations_Live.py

Contents of algo directory:

HCA_Fixed_Ratio_Allocations.ipynb
HCA_Fixed_Ratio_Allocations_Live.py
HCA_Fixed_Ratio_Allocations.pkl
HCA_Fixed_Ratio_Allocations.py
HCA_Fixed_Ratio_Allocations_tearsheet.ipynb


## Zipline backtest: 

- Method: Jupyter magic cell (%%) 
- Execution of zipline code in cell containing command line command
- Uncomment first line and hit (shift-enter) inside the cell to run simulation backtest

In [6]:
 %%zipline --start=2018-1-1 --end=2020-10-02 -b hca-symbol -o $ALGO_BT_OUT

# Source: adapted from various algos on quantopian
# HCA Conversion Date: 08-13-2020
# Conversion Author: Anthony garner

# Simple rebalanced portfolio


import matplotlib.pyplot as plt

from zipline.api import (symbol, symbols, date_rules, time_rules, order_target_percent, 
                         record, schedule_function, get_datetime)
from trading_calendars import get_calendar


def initialize(context):
    schedule_function(func=trade,date_rule=date_rules.every_day(),time_rule=time_rules.market_open(),half_days=True)
    context.asserts = symbols('SPY','SHY','TLT','GLD')
    context.asserts_position = [0.25, 0.25,0.25,0.25]
    context.rebalance_inteval = 'Q'#'Q', #'D', #'M' #'Q' #'Y'
    context.rebalance_date = 0
    context.fired = False


def handle_data(context, data):
    record(SPY=data[symbol('SPY')].price)


def is_new_day(context, now):
    return ( (now.year > context.rebalance_date.year) or (now.month > context.rebalance_date.month) or((now.month == context.rebalance_date.month) and (now.day > context.rebalance_date.day)))             


def is_new_month(context, now):
    return ((now.year > context.rebalance_date.year) or ((now.year == context.rebalance_date.year) and (now.month > context.rebalance_date.month)))


def is_new_quarter(context, now):
    return ((now.year > context.rebalance_date.year) or ((now.year == context.rebalance_date.year) and (now.month == context.rebalance_date.month + 3)))


def is_new_year(context, now):
    return (now.year > context.rebalance_date.year)


def need_rebalance(context, now):
    return ((context.rebalance_inteval == 'Y' and is_new_year(context, now))or 
           (context.rebalance_inteval == 'Q' and is_new_quarter(context, now)) or 
           (context.rebalance_inteval == 'M' and is_new_month(context, now)) or 
           (context.rebalance_inteval == 'D' and is_new_day(context, now)))


def init_portfolio(context, data):
    for i in range(0, len(context.asserts)):
        if data.can_trade(context.asserts[i]):
            #log.debug("rebalance " + context.asserts[i].symbol + " to:" + str(context.asserts_position[i]*100) + "%")
            order_target_percent(context.asserts[i], context.asserts_position[i])


def rebalance(context, data):
    for i in range(0, len(context.asserts)):
        if data.can_trade(context.asserts[i]):
            #log.debug("rebalance " + context.asserts[i].symbol + " to:" + str(context.asserts_position[i]*100) + "%")
            order_target_percent(context.asserts[i], context.asserts_position[i])
        

# Will be called on every trade event for the securities you specify. 
def trade(context, data):
    if not context.fired:
        context.rebalance_date = get_datetime()
        #log.info("build portfolio at " + str(context.rebalance_date))
        init_portfolio(context, data)
        context.fired = True
        now = get_datetime()
    else:
        now = get_datetime()
        if (need_rebalance(context, now)):
            #log.info("new rebalance arrivied:" + str(now))
            rebalance(context, data)
            context.rebalance_date = now


def analyze(context, perf):
    ax1 = plt.subplot(211)
    perf.portfolio_value.plot(ax=ax1)
    ax2 = plt.subplot(212, sharex=ax1)
    perf.SPY.plot(ax=ax2)
    plt.gcf().set_size_inches(18, 8)
    plt.show()

extension: hca_root_path = /home/hca-ws002/hca
extension:TODAY_STR = 2020-12-06
extension:TWO_YR_AGO_STR = 2018-12-06
extension:  start_date=2018-12-06 end_date = 2020-12-06


SPY  algo_volatility  algorithm_period_return  \
2018-01-02 21:00:00+00:00  268.77              NaN                 0.000000   
2018-01-03 21:00:00+00:00  270.47         0.005713                -0.000509   
2018-01-04 21:00:00+00:00  271.61         0.022682                 0.001671   
2018-01-05 21:00:00+00:00  273.42         0.018557                 0.002377   
2018-01-08 21:00:00+00:00  273.92         0.016240                 0.002643   
2018-01-09 21:00:00+00:00  274.54         0.032790                -0.001375   
2018-01-10 21:00:00+00:00  274.12         0.029964                -0.001376   
2018-01-11 21:00:00+00:00  276.12         0.035012                 0.002230   
2018-01-12 21:00:00+00:00  277.92         0.041468                 0.007329   
2018-01-16 21:00:00+00:00  276.97         0.039121                 0.007871   
2018-01-17 21:00:00+00:00  279.61         0.037319                 0.007841   
2018-01-18 21:00:00+00:00  279.14         0.040139                 0.004474   
2018-01-19 21:00:00+00:00  280.41         0.038559                 0.005568   
2018-01-22 21:00:00+00:00  282.69         0.038262                 0.008268   
2018-01-23 21:00:00+00:00  283.29         0.038166                 0.011290   
2018-01-24 21:00:00+00:00  283.18         0.037018                 0.012889   
2018-01-25 21:00:00+00:00  283.30         0.035887                 0.013235   
2018-01-26 21:00:00+00:00  286.58         0.035197                 0.015420   
2018-01-29 21:00:00+00:00  284.68         0.040172                 0.010412   
2018-01-30 21:00:00+00:00  281.76         0.044048                 0.005193   
2018-01-31 21:00:00+00:00  281.90         0.044159                 0.008456   
2018-02-01 21:00:00+00:00  281.58         0.044670                 0.005360   
2018-02-02 21:00:00+00:00  275.45         0.057793                -0.005899   
2018-02-05 21:00:00+00:00  263.93         0.061112                -0.013276   
2018-02-06 21:00:00+00:00  269.13         0.059920                -0.012770   
2018-02-07 21:00:00+00:00  267.67         0.060390                -0.017756   
2018-02-08 21:00:00+00:00  257.63         0.064873                -0.026944   
2018-02-09 21:00:00+00:00  261.50         0.064273                -0.025049   
2018-02-12 21:00:00+00:00  265.34         0.066160                -0.019363   
2018-02-13 21:00:00+00:00  266.00         0.065942                -0.016275   
...                           ...              ...                      ...   
2020-08-21 20:00:00+00:00  339.48         0.070337                 0.343780   
2020-08-24 20:00:00+00:00  342.92         0.070284                 0.344780   
2020-08-25 20:00:00+00:00  344.12         0.070234                 0.343990   
2020-08-26 20:00:00+00:00  347.57         0.070232                 0.350413   
2020-08-27 20:00:00+00:00  348.33         0.070309                 0.341637   
2020-08-28 20:00:00+00:00  350.58         0.070343                 0.349870   
2020-08-31 20:00:00+00:00  349.31         0.070292                 0.351610   
2020-09-01 20:00:00+00:00  352.60         0.070306                 0.358964   
2020-09-02 20:00:00+00:00  357.70         0.070267                 0.362640   
2020-09-03 20:00:00+00:00  345.39         0.070548                 0.348014   
2020-09-04 20:00:00+00:00  342.57         0.070609                 0.339800   
2020-09-08 20:00:00+00:00  333.21         0.070674                 0.331488   
2020-09-09 20:00:00+00:00  339.79         0.070735                 0.340812   
2020-09-10 20:00:00+00:00  333.89         0.070739                 0.335194   
2020-09-11 20:00:00+00:00  334.06         0.070687                 0.336108   
2020-09-14 20:00:00+00:00  338.46         0.070700                 0.343342   
2020-09-15 20:00:00+00:00  340.17         0.070648                 0.343495   
2020-09-16 20:00:00+00:00  338.82         0.070602                 0.342200   
2020-09-17 20:00:00+00:00  335.84         0.070582                 0.338099   
2

## Display your current bundles

In [7]:
!zipline bundles # Finds all bundles

/home/hca-dev/miniconda3/envs/hca/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/home/hca-dev/miniconda3/envs/hca/lib/python3.6/site-packages/odo/backends/pandas.py:94: FutureWarning: pandas.tslib is deprecated and will be removed in a future version.
You can access NaTType as type(pandas.NaT)
  @convert.register((pd.Timestamp, pd.Timedelta), (pd.tslib.NaTType, type(None)))
extension: hca_root_path = /home/hca-ws002/hca
extension:TODAY_STR = 2020-12-06
extension:TWO_YR_AGO_STR = 2018-12-06
extension:  start_date=2018-12-06 end_date = 2020-12-06
csvdir <no ingestions>
hca-symbol 2020-11-20 16:06:36.135842
iex <no ingestions>
quandl <no ingestions>
quantopian-quandl <no ingestions>
sharadar-eqfd 2020-12-06 16:35:39.030779
sharadar-funds 2020-12-

## Ingest Sharadar funds assets for today

In [8]:
# Ingest Sharadar funds assets for today, if needed.

# Only need to ingest Funds for this algo, and this takes less processing time and system memory than ingesting
# all of Sharadar Equities plus Funds bundle (sharadar-eqfd)

#!zipline ingest -b sharadar-funds

# Yahoo Finance Free Data
# Trend_Following
# !export YAHOO_SYM_LST=SPY,ZSL,KOLD,GLD,SHY;zipline ingest -b yahoo_direct
# Fixed_Ratio_Allocations
!export YAHOO_SYM_LST=QQQ,XLP,IEF,TLT;zipline ingest -b yahoo_direct


/home/hca-dev/miniconda3/envs/hca/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/home/hca-dev/miniconda3/envs/hca/lib/python3.6/site-packages/odo/backends/pandas.py:94: FutureWarning: pandas.tslib is deprecated and will be removed in a future version.
You can access NaTType as type(pandas.NaT)
  @convert.register((pd.Timestamp, pd.Timedelta), (pd.tslib.NaTType, type(None)))
extension: hca_root_path = /home/hca-ws002/hca
extension:TODAY_STR = 2020-12-06
extension:TWO_YR_AGO_STR = 2018-12-06
extension:  start_date=2018-12-06 end_date = 2020-12-06
[2020-12-06 23:35:16.144338] INFO: zipline.data.bundles.ingester: symbols are: ('QQQ', 'XLP', 'TLT', 'IEF')
[2020-12-06 23:35:16.145214] INFO: zipline.data.bundles.ingester: writing data...
            

## Zipline backtest, alternative method

- Method: command line
- Execution of zipline code, located in a file, using below command line execution with magic (`!`) invocation
- This line can also be run in a terminal by copying everything past the `!` and pasting (shift-insert) it into the target terminal

In [9]:
# !zipline run -f $ALGO_BT  --start=2018-1-1 --end=2020-10-02 -b sharadar-funds -o $ALGO_BT_OUT

## Run Zipline live on IB-TWS via command line

- Method: command line
- Execution of zipline code using below command line execution using magic (`!`) invocation

**Notes:** 
- IB-TWS or IB-Gateway must be running, with `IB_ACCT` and `IB_URI` port being correct to live trade
- Change `I_WANT_TO_RUN_THIS_CODE` to `True` below to run zipline live on IB-TWS/IB-Gateway

In [10]:
TODAY = pd.datetime.today().strftime("%Y-%m-%d")
print("TODAY = {}".format(TODAY))

TODAY = 2020-12-06


In [ ]:
ALGO_STATE = ALGO_PATH + "strategy.state" 
ALGO_RTB   = ALGO_PATH + "realtime-bars/"

# Edit the following URI to match your IB account and port info.
BROKER = "ib"
IB_ACCT = "DU1568488"
IB_URI = "127.0.0.1:7497:1301"

#  PRICES_BUNDLE = "hca-symbol" #Used for Zipline Symbols to Live Broker Mapping(e.g. IB)
PRICES_BUNDLE = "yahoo_direct" #Used for free data bundle for zipline.
DATA_FREQ = "daily"

# Change following to 'True' and run cell (control-enter) to execute live run.
I_WANT_TO_RUN_THIS_CODE = True
# I_WANT_TO_RUN_THIS_CODE = True

if I_WANT_TO_RUN_THIS_CODE:
    
    !zipline run \
        -s $TODAY \
        -f $ALGO_LIVE \
        --bundle $PRICES_BUNDLE \
        --broker $BROKER \
        --broker-uri $IB_URI \
        --broker-acct $IB_ACCT \
        --data-frequency $DATA_FREQ \
        --state-file $ALGO_STATE \
        --realtime-bar-target $ALGO_RTB 

/home/hca-dev/miniconda3/envs/hca/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/home/hca-dev/miniconda3/envs/hca/lib/python3.6/site-packages/odo/backends/pandas.py:94: FutureWarning: pandas.tslib is deprecated and will be removed in a future version.
You can access NaTType as type(pandas.NaT)
  @convert.register((pd.Timestamp, pd.Timedelta), (pd.tslib.NaTType, type(None)))
extension: hca_root_path = /home/hca-ws002/hca
extension:TODAY_STR = 2020-12-06
extension:TWO_YR_AGO_STR = 2018-12-06
extension:  start_date=2018-12-06 end_date = 2020-12-06
[2020-12-06 23:35:19.904980] INFO: IB Broker: Connecting: 127.0.0.1:7497:1301
[2020-12-06 23:35:19.905301] ERROR: IB Broker: [502] Couldn't connect to TWS.  Confirm that "Enable ActiveX and Socket Clien

In [ ]:
# This bundle is write-protected, and not to be updated. It will produce a write error.
# Used only as a zipline-to-live-broker symbol mapping 
#  and for older backtests dating from the bundle production date(2020-12-06T23;31;54.306657)
#! zipline ingest -b hca-symbol